In [ ]:
# !pacman -S firefox firefox-i18n-r  geckodriver
# !pip install selenium

In [23]:
!rm -v geckodriver.log_
!mv -v geckodriver.log geckodriver.log_

удалён 'geckodriver.log_'
mv: не удалось выполнить stat для 'geckodriver.log': Нет такого файла или каталога


In [ ]:
import re
from tqdm.notebook import tqdm
import pandas as pd
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.options import FirefoxProfile

In [ ]:
# url = (
# 'https://www.avito.ru/yaroslavskaya_oblast/avtomobili/chevrolet/niva'
# '?bt=0'
# '&i=1'
# '&pmax=150000'
# '&pmin=10000'
# )

url = 'https://www.avito.ru/sevastopol/kvartiry/prodam?'# '&pmax=5000000'

profile_path = '/home/mechanoid/.mozilla/firefox/p144xo2m.default-release'

In [87]:
def get_pages_count(root):
    pp = re.sub( r'.*?p=', '', root.find_all('a',{'class':'pagination-page'})[-1].attrs['href'] ) 
    return 1 if not re.match(r'\d{1,3}', pp) else int(pp)


def parse_item(tag):
    try:
        return {    
    'avito_id':tag.attrs['data-item-id'],   # https://www.avito.ru/<id>
    'title':tag.find('a',attrs={'itemprop':'url'}).attrs['title'],    
    'price':tag.find('meta',attrs={'itemprop':'price'}).attrs['content'],
     # 'cur':tag.find('meta',attrs={'itemprop':'priceCurrency'}).attrs['content'],
    'adr':tag.find('div',attrs={'data-marker':'item-address'}).text,
    'description':tag.find('meta',attrs={'itemprop':'description'}).attrs['content'],
        }
    except:
        return dict()


options=Options()
options.profile = FirefoxProfile(profile_path) 
options.headless = True
driver = webdriver.Firefox( options=options)

driver.get(url)
html = driver.page_source

root = BeautifulSoup(html)
pages = get_pages_count(root)
print(pages)
data = [ parse_item(tag) for tag in root.find_all('div',{'data-marker':'item'}) ]

for p in tqdm(range(2,pages)):
    driver.get(url+f'&p={p}')
    html = driver.page_source
    root = BeautifulSoup(html)
    data.extend( [ parse_item(tag) for tag in root.find_all('div',{'data-marker':'item'}) ] )

# driver.close()
driver.quit()

/tmp/ipykernel_508904/2340081842.py:45: DeprecationWarning: firefox_profile has been deprecated, please use an Options object
  options.profile = FirefoxProfile(profile_path)
/tmp/ipykernel_508904/2340081842.py:45: DeprecationWarning: Setting a profile has been deprecated. Please use the set_preference and install_addons methods
  options.profile = FirefoxProfile(profile_path)


81


  0%|          | 0/79 [00:00<?, ?it/s]

In [112]:
import pandas as pd
df = pd.DataFrame(data).dropna()
print(len(df))
df.sample(10)

3949


,title,description,price,cur,adr,avito_id
3906,"Объявление «2-к. квартира, 58,1 м², 9/9 эт.» 7...",Продается 2-х комнатная квартира в самом прекр...,11400000,RUB,"ЖК «Апельсин» ул. Павла Корчагина, д. 23, блок...",2406535369
1882,"Объявление «1-к. апартаменты, 27,5 м², 2/6 эт....","Продаются отличные апартаменты, которые распол...",3800000,RUB,"Фиолентовское ш., 134к5",2424886339
1161,"Объявление «3-к. квартира, 77 м², 2/8 эт.» 17 ...",В продаже большая трехкомнатная двухуровневая ...,12500000,RUB,"ул. Адмирала Юмашева, 13",2331971549
3029,"Объявление «3-к. квартира, 71 м², 3/5 эт.» 17 ...",Продаётся уютная трёхкомнатная квартира в Гага...,10000000,RUB,"ул. Александра Маринеско, 11",2446061678
655,"Объявление «1-к. квартира, 36,3 м², 3/9 эт.» с...",Крупнейший в Севастополе семейный микрорайон «...,5808000,RUB,"ЖК «Доброгород»ул. Токарева, 7 этап, блок-секц...",2505695205
3520,"Объявление «1-к. квартира, 30,3 м², 4/5 эт.» 6...",Однокомнатная квартира 30.3м2 на 4/5 этаже дом...,7100000,RUB,"ул. Меньшикова, 19",2455862087
2518,"Объявление «2-к. квартира, 44,7 м², 5/5 эт.» 1...",В продаже двухкомнатная уютная квартира в тихо...,5800000,RUB,"ул. Маршала Бирюзова, 9",2281182173
670,"Объявление «1-к. квартира, 28,3 м², 3/9 эт.» с...",Крупнейший в Севастополе семейный микрорайон «...,4952500,RUB,"ЖК «Доброгород»ул. Токарева, 7 этап, блок-секц...",2505809392
1853,"Объявление «2-к. квартира, 45 м², 5/5 эт.» 16 ...","Двухкомнатная квартира 45,3 м2, на 5/5 этаже д...",7000000,RUB,"пр-т Генерала Острякова, 94",2425679365
231,"Объявление «1-к. квартира, 30,6 м², 3/4 эт.» 8...",Срочно! Собственник! Продаю 1 — комнатную уютн...,4950000,RUB,"ул. Героев Севастополя, 23",2464975481


In [114]:
from datetime import datetime as dt

ts = dt.now().strftime('%Y-%m-%d_%H-%M')
df[['avito_id','title','price','adr','description',]].to_excel(f'data/avito_{ts}_raw.xlsx',index=False)

ts

'2022-08-26_17-29'